# Preprocessing of RNA-seq counts
Reads should be processed according to the manuscript using Salmon

This script will generate normalized counts that will be used by further analysis

In [2]:
library('DESeq2')
library('tidyverse')
library('readxl')
library("RColorBrewer")
library("pheatmap")
options(bitmapType="cairo")
library("tximeta")
library("BiocParallel")
library("AnnotationDbi")
library("org.Hs.eg.db")
register(MulticoreParam(16))
setwd("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/RNA_seq_analysis")

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGe

In [3]:
# location of Salom counts
directory = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_RNA/salmon_counts"

design = read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/metadata/cleaned_RNA_metadata.csv")

New names:
• `` -> `...1`
Rows: 128 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): proper_name, sample, patient, cell_type, condition, group
dbl (23): ...1, age, female_sex, year_psa_onset, year_psoriasis_onset, disea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# assume files are named accordingly to the following pattern. check that files exist
design$files <- file.path(directory, paste0(design$sample, "_quant"), "quant.sf")
design <- design %>% mutate(names = sample)
file.exists(design$files)

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [31] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [46] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [61] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [76] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [91] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[106] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[121] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [5]:
se <- tximeta(design)

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 
61 
62 
63 
64 
65 
66 
67 
68 
69 
70 
71 
72 
73 
74 
75 
76 
77 
78 
79 
80 
81 
82 
83 
84 
85 
86 
87 
88 
89 
90 
91 
92 
93 
94 
95 
96 
97 
98 
99 
100 
101 
102 
103 
104 
105 
106 
107 
108 
109 
110 
111 
112 
113 
114 
115 
116 
117 
118 
119 
120 
121 
122 
123 
124 
125 
126 
127 
128 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 29 ]

loading existing TxDb created: 2022-03-21 12:43:36

Loading required package: GenomicFeatures

loading existing transcript ranges created: 2022-03-21 12:43:40



In [6]:
gse <- summarizeToGene(se)
gse$active_disease <- factor(gse$active_disease)
gse$female_sex <- factor(gse$female_sex)
gse$on_bDMARD_tsDMARD <- factor(gse$on_bDMARD_tsDMARD)
gse$on_csDMARD <- factor(gse$on_csDMARD)
gse$on_steroid <- factor(gse$on_steroid)
gse$On_MTX <- factor(gse$On_MTX)
gse$group <- factor(paste0(gse$active_disease, gse$on_bDMARD_tsDMARD))

loading existing TxDb created: 2022-03-21 12:43:36

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2022-03-21 12:43:58

summarizing abundance

summarizing counts

summarizing length



In [7]:
# generate DESeq2 dataset
dds <- DESeqDataSet(gse, design = ~ cell_type + female_sex)

using counts and average transcript lengths from tximeta

Warning message in DESeqDataSet(gse, design = ~cell_type + female_sex):
"some variables in design formula are characters, converting to factors"


In [8]:
keep <- rowMeans(counts(dds)) >= 5
dds <- dds[keep,]
vsd <- vst(dds, blind = TRUE)

dds <- DESeq(dds)

using 'avgTxLength' from assays(dds), correcting for library size

estimating size factors

using 'avgTxLength' from assays(dds), correcting for library size

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 240 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [9]:
# add annotation information and save

counts_total <- as.data.frame(counts(dds,normalized=TRUE))
counts_total <- rownames_to_column(counts_total, var = "ensembl")
counts_total <- counts_total %>% separate(ensembl, c("ENSG",NA),"\\.", remove = FALSE )
counts_total$symbol <- mapIds(org.Hs.eg.db,
                     keys=pull(counts_total, ENSG),
                     column="SYMBOL",
                     keytype="ENSEMBL",
                     multiVals="first")
counts_total$genename <- mapIds(org.Hs.eg.db,
                     keys=pull(counts_total, ENSG),
                     column="GENENAME",
                     keytype="ENSEMBL",
                     multiVals="first")
counts_total$entrez = mapIds(org.Hs.eg.db,
                     keys=pull(counts_total, ENSG),
                     column="ENTREZID",
                     keytype="ENSEMBL",
                     multiVals="first")
                     
write.csv(counts_total,file="RNA_normalized_counts.csv")

write.csv(assay(vsd),file="RNA_vsd_counts.csv")


'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns



In [22]:
pcaData <- plotPCA(vsd, intgroup=c("cell_type", "female_sex","patient"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
p <- ggplot(pcaData, aes(PC1, PC2, color=cell_type, shape=female_sex)) +
  geom_point(size=3) +
  geom_text(aes(label=patient), hjust=1.1, vjust=1.1) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()

  ggsave("DESeq2_PCA_all_samples_RNA.png", p, width = 15, height = 15, units = "in")

In [3]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Scientific Linux 7.4 (Nitrogen)

Matrix products: default
BLAS:   /opt/gridware/el7/apps/gcc/R/4.1.2/lib64/R/lib/libRblas.so
LAPACK: /opt/gridware/el7/apps/gcc/R/4.1.2/lib64/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] org.Hs.eg.db_3.14.0         AnnotationDbi_1.56.2       
 [3] BiocParallel_1.28.3         tximeta_1.12.4             
 [5] pheatmap_1.0.12             RColorBrewer_1.1-3         
 [7] readxl_1.3.1                forca